In [0]:
data = spark.read.format('libsvm').load('/FileStore/tables/sample_kmeans_data.txt')

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0| (3,[],[])|
 1.0|(3,[0,1,2],[0.1,0...|
 2.0|(3,[0,1,2],[0.2,0...|
 3.0|(3,[0,1,2],[9.0,9...|
 4.0|(3,[0,1,2],[9.1,9...|
 5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+

In [0]:
final_data = data.select('features')

In [0]:
kmeans = KMeans().setK(2).setSeed(123)

In [0]:
model = kmeans.fit(final_data)

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator
predictions = model.transform(final_data)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.9997530305375207

In [0]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[9.1 9.1 9.1]
[0.1 0.1 0.1]

In [0]:
predictions.show()  # unsupervised learning, we put all data, not test_train split

+--------------------+----------+
 features|prediction|
+--------------------+----------+
 (3,[],[])| 1|
(3,[0,1,2],[0.1,0...| 1|
(3,[0,1,2],[0.2,0...| 1|
(3,[0,1,2],[9.0,9...| 0|
(3,[0,1,2],[9.1,9...| 0|
(3,[0,1,2],[9.2,9...| 0|
+--------------------+----------+

In [0]:
dataset = spark.read.csv("/FileStore/tables/seeds_dataset.csv", header = True, inferSchema = True)

In [0]:
display(dataset.head(1))

area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_of_groove
15.26,14.84,0.871,5.763,3.312,2.221,5.22


In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
dataset.columns

Out[20]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [0]:
assembler = VectorAssembler(inputCols = dataset.columns, outputCol = 'features')

In [0]:
final_data = assembler.transform(dataset)

In [0]:
final_data.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
# they don't mind extra columns
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol = 'features', outputCol = 'scaled_features')

In [0]:
final_data = scaler.fit(final_data).transform(final_data)

In [0]:
final_data .show(1)

+-----+---------+-----------+----------------+---------------+---------------------+----------------+--------------------+--------------------+
 area|perimeter|compactness|length_of_kernel|width_of_kernel|asymmetry_coefficient|length_of_groove| features| scaled_features|
+-----+---------+-----------+----------------+---------------+---------------------+----------------+--------------------+--------------------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|[15.26,14.84,0.87...|[5.24452795332028...|
+-----+---------+-----------+----------------+---------------+---------------------+----------------+--------------------+--------------------+
only showing top 1 row

In [0]:
kmeans = KMeans(featuresCol = 'scaled_features', k = 3)

In [0]:
model = kmeans.fit(final_data)

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator
predictions = model.transform(final_data)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.616267393520126

In [0]:
model.transform(final_data).select('prediction').show()

+----------+
prediction|
+----------+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 1|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 2|
+----------+
only showing top 20 rows

In [0]:
data = spark.read.csv("/FileStore/tables/hack_data.csv", header = True, inferSchema = True)

In [0]:
display(data.head(1))

Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
8.0,391.09,1,2.96,7.0,Slovenia,72.37


In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

Out[50]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
data = data.drop('Location')

In [0]:
data.columns

Out[52]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [0]:
assembler = VectorAssembler(inputCols = data.columns, outputCol = 'features')

In [0]:
final_data = assembler.transform(data)

In [0]:
final_data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol = 'features', outputCol = 'scaled_features')

In [0]:
scaler_model = scaler.fit(final_data).transform(final_data)

In [0]:
scaler_model.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)
-- scaled_features: vector (nullable = true)

In [0]:
Kmeans2 = KMeans(featuresCol = 'scaled_features', k = 2)
Kmeans3 = KMeans(featuresCol = 'scaled_features', k = 3)

In [0]:
model_k2 = Kmeans2.fit(scaler_model)
model_k3 = Kmeans3.fit(scaler_model)

In [0]:
model_k3.transform(scaler_model).select('prediction').groupby('prediction').count().show()  # not even count

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+

In [0]:
# even count, so we can understand there're two hackers.
model_k2.transform(scaler_model).select('prediction').groupby('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+